# Logistic Regression — Enhanced (Translated & Optimized)

**Original source:** [Jack-Cherish/Machine-Learning](https://github.com/Jack-Cherish/Machine-Learning)  
**Translated from Chinese and enhanced with advanced statistical analysis.**

## What this notebook covers

| Section | Content |
|---|---|
| 1 | Gradient ascent demo (find maximum of f(x) = −x² + 4x) |
| 2 | Logistic regression on 2D synthetic data (batch GD + L2 regularization) |
| 3 | Horse colic survival prediction (UCI dataset, 21 clinical features) |
| 4 | Correlation matrix → interaction term selection |
| 5 | Custom SGD vs sklearn — full metric comparison |
| 6 | Stratified k-fold cross-validation |

## Enhancements over the original
- Full English translation of all code, comments, and output
- Feature standardization (zero mean, unit variance) — prevents gradient scale issues
- L2 regularization in batch gradient ascent
- Interaction terms derived from correlation matrix analysis
- Comprehensive evaluation: confusion matrix, accuracy, precision, recall, specificity, F1, AUC-ROC, log-loss, MCC
- Convergence monitoring via log-loss curve
- Stratified k-fold cross-validation with mean ± std reporting

## Imports

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    confusion_matrix, classification_report, roc_auc_score,
    log_loss, matthews_corrcoef, roc_curve
)

%matplotlib inline
plt.rcParams['figure.dpi'] = 110

---
## Part 1 — Gradient Ascent Demo

Illustrates the gradient ascent update rule by finding the maximum of:

$$f(x) = -x^2 + 4x$$

The derivative $f'(x) = -2x + 4 = 0$ at $x = 2$, so we expect the algorithm to converge near 2.

In [ ]:
def gradient_ascent_demo():
    """Find the maximum of f(x) = -x^2 + 4x using gradient ascent."""
    def f_prime(x):
        return -2 * x + 4

    x_old = -1.0          # initial guess (must differ from x_new to enter loop)
    x_new = 0.0           # starting point
    learning_rate = 0.01  # controls update magnitude
    precision = 1e-8      # convergence threshold

    iterations = 0
    while abs(x_new - x_old) > precision:
        x_old = x_new
        x_new = x_old + learning_rate * f_prime(x_old)
        iterations += 1

    print(f"Converged in {iterations} iterations")
    print(f"Maximum of f(x) = -x² + 4x found at x = {x_new:.8f}  (expected: 2.0)")
    print(f"f({x_new:.4f}) = {-x_new**2 + 4*x_new:.6f}  (expected: 4.0)")

gradient_ascent_demo()

---
## Part 2 — Logistic Regression on 2D Synthetic Data

Shared helper functions used throughout the notebook.

In [ ]:
# ── Sigmoid ──────────────────────────────────────────────────────────────────

def sigmoid(z):
    """Numerically stable sigmoid: avoids overflow for large |z|."""
    return np.where(z >= 0,
                    1.0 / (1.0 + np.exp(-z)),
                    np.exp(z) / (1.0 + np.exp(z)))


# ── Evaluation suite ─────────────────────────────────────────────────────────

def evaluate(y_true, y_pred, y_prob, model_name="Model", class_names=None):
    """Print a comprehensive suite of classification metrics.

    Reports: confusion matrix, accuracy, precision, recall, specificity,
    F1, AUC-ROC, log-loss, Matthews Correlation Coefficient.
    """
    if class_names is None:
        class_names = ["Class 0", "Class 1"]

    print(f"\n{'='*60}")
    print(f"  Evaluation — {model_name}")
    print(f"{'='*60}")

    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()

    print(f"\nConfusion Matrix:")
    print(f"              Predicted 0   Predicted 1")
    print(f"  Actual 0       {tn:6d}        {fp:6d}")
    print(f"  Actual 1       {fn:6d}        {tp:6d}")

    accuracy    = (tp + tn) / (tp + tn + fp + fn)
    precision   = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall      = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0
    f1          = (2 * precision * recall / (precision + recall)
                   if (precision + recall) > 0 else 0.0)
    mcc  = matthews_corrcoef(y_true, y_pred)
    auc  = roc_auc_score(y_true, y_prob)
    ll   = log_loss(y_true, y_prob)

    print(f"\nCore Metrics:")
    print(f"  Accuracy    : {accuracy:.4f}  ({(1-accuracy)*100:.2f}% error rate)")
    print(f"  Precision   : {precision:.4f}  (TP / (TP+FP))")
    print(f"  Recall      : {recall:.4f}  (sensitivity, TP / (TP+FN))")
    print(f"  Specificity : {specificity:.4f}  (TN / (TN+FP))")
    print(f"  F1-Score    : {f1:.4f}")
    print(f"\nAdvanced Metrics:")
    print(f"  AUC-ROC     : {auc:.4f}")
    print(f"  Log-Loss    : {ll:.4f}")
    print(f"  MCC         : {mcc:.4f}  (ranges -1 to +1, 0 = random)")
    print(f"\nFull Classification Report:")
    print(classification_report(y_true, y_pred, target_names=class_names))

    return dict(accuracy=accuracy, precision=precision, recall=recall,
                specificity=specificity, f1=f1, auc_roc=auc,
                log_loss=ll, mcc=mcc)


print("Helper functions loaded.")

### 2.1 Load the 2D synthetic dataset (`testSet.txt`)

In [ ]:
def load_2d_dataset(filepath="testSet.txt"):
    """Load 2D binary dataset.  Format: x1  x2  label (space-separated).
    Prepends a bias column of 1.0.
    """
    data_matrix, labels = [], []
    with open(filepath) as fh:
        for line in fh.readlines():
            parts = line.strip().split()
            data_matrix.append([1.0, float(parts[0]), float(parts[1])])
            labels.append(int(parts[2]))
    return np.array(data_matrix), np.array(labels)


def standardize_2d(X):
    """Standardize feature columns (skip bias at index 0)."""
    arr = X.copy().astype(float)
    means = arr[:, 1:].mean(axis=0)
    stds  = arr[:, 1:].std(axis=0)
    stds[stds == 0] = 1.0
    arr[:, 1:] = (arr[:, 1:] - means) / stds
    return arr, means, stds


X_2d, y_2d = load_2d_dataset("testSet.txt")
X_2d_std, _, _ = standardize_2d(X_2d)

print(f"Dataset shape : {X_2d.shape}")
print(f"Class balance : {(y_2d==0).sum()} class-0  /  {(y_2d==1).sum()} class-1")

# Scatter plot
fig, ax = plt.subplots(figsize=(6, 4))
ax.scatter(X_2d[y_2d==1, 1], X_2d[y_2d==1, 2], s=25, c='red',   marker='s', alpha=0.7, label='Class 1')
ax.scatter(X_2d[y_2d==0, 1], X_2d[y_2d==0, 2], s=25, c='green',             alpha=0.7, label='Class 0')
ax.set_title('2D Synthetic Dataset'); ax.set_xlabel('X1'); ax.set_ylabel('X2'); ax.legend()
plt.tight_layout(); plt.show()

### 2.2 Batch gradient ascent with L2 regularization

Maximises the regularized log-likelihood:

$$\mathcal{L}(\mathbf{w}) = \sum_i \left[ y_i \log h_i + (1-y_i) \log(1-h_i) \right] - \frac{\lambda}{2} \|\mathbf{w}\|^2$$

In [ ]:
def batch_gradient_ascent(X, y, learning_rate=0.005, max_iter=1000, l2_lambda=0.01):
    """Batch gradient ascent with optional L2 regularization.

    Parameters
    ----------
    X           : array (m, n) — includes bias column
    y           : array (m,)
    learning_rate : float
    max_iter    : int
    l2_lambda   : float — L2 penalty strength (0 = disabled)

    Returns
    -------
    weights      : ndarray (n,)
    loss_history : list[float]
    """
    Xm = np.mat(X, dtype=float)
    ym = np.mat(y, dtype=float).T
    m, n = Xm.shape
    w = np.ones((n, 1))
    loss_history = []

    for _ in range(max_iter):
        h = sigmoid(Xm * w)
        error = ym - h
        penalty = np.zeros_like(w)
        penalty[1:] = l2_lambda * w[1:]          # do not penalize bias
        w = w + learning_rate * (Xm.T * error - penalty)

        h_arr = np.asarray(h).ravel()
        y_arr = np.asarray(ym).ravel()
        loss_history.append(log_loss(y_arr, h_arr))

    return np.asarray(w).ravel(), loss_history


weights_batch, loss_hist_batch = batch_gradient_ascent(
    X_2d_std, y_2d, learning_rate=0.005, max_iter=1000, l2_lambda=0.01
)
print(f"Final weights: {weights_batch}")

### 2.3 Convergence curve

In [ ]:
plt.figure(figsize=(7, 3))
plt.plot(loss_hist_batch, color='steelblue', lw=1.5)
plt.xlabel('Iteration'); plt.ylabel('Log-Loss')
plt.title('Convergence — Batch Gradient Ascent (L2 regularized)')
plt.tight_layout(); plt.show()

### 2.4 Evaluation + decision boundary

In [ ]:
y_prob_batch = sigmoid(X_2d_std @ weights_batch)
y_pred_batch = (y_prob_batch >= 0.5).astype(int)

metrics_batch = evaluate(y_2d, y_pred_batch, y_prob_batch,
                         model_name="Logistic Regression — Batch GD (2D)")

In [ ]:
# Decision boundary: w0 + w1*x1 + w2*x2 = 0  =>  x2 = -(w0 + w1*x1) / w2
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Left: decision boundary
ax = axes[0]
ax.scatter(X_2d_std[y_2d==1, 1], X_2d_std[y_2d==1, 2], s=25, c='red',   marker='s', alpha=0.7, label='Class 1')
ax.scatter(X_2d_std[y_2d==0, 1], X_2d_std[y_2d==0, 2], s=25, c='green',             alpha=0.7, label='Class 0')
x1r = np.linspace(X_2d_std[:, 1].min()-0.3, X_2d_std[:, 1].max()+0.3, 200)
x2b = -(weights_batch[0] + weights_batch[1] * x1r) / weights_batch[2]
ax.plot(x1r, x2b, 'b-', lw=2, label='Decision boundary')
ax.set_title('Decision Boundary (Standardized)'); ax.set_xlabel('X1'); ax.set_ylabel('X2'); ax.legend()

# Right: ROC curve
fpr, tpr, _ = roc_curve(y_2d, y_prob_batch)
auc_val = roc_auc_score(y_2d, y_prob_batch)
axes[1].plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC (AUC={auc_val:.3f})')
axes[1].plot([0,1],[0,1],'k--', lw=1, label='Random')
axes[1].set_xlabel('False Positive Rate'); axes[1].set_ylabel('True Positive Rate')
axes[1].set_title('ROC Curve — Batch GD'); axes[1].legend(loc='lower right')

plt.tight_layout(); plt.show()

---
## Part 3 — Horse Colic Survival Prediction (UCI Dataset)

Binary classification on 21 clinical veterinary features.  
**Label:** 0 = did not survive, 1 = survived.

### 3.1 Load data

In [ ]:
# Feature names translated from original UCI documentation
FEATURE_NAMES = [
    "surgery",                   # 1=yes, 2=no
    "age",                       # 1=adult, 2=young
    "rectal_temp",               # degrees Celsius
    "pulse",                     # beats per minute
    "respiratory_rate",          # breaths per minute
    "temp_extremities",          # 1=normal, 2=warm, 3=cool, 4=cold
    "peripheral_pulse",          # 1=normal, 2=increased, 3=reduced, 4=absent
    "mucous_membranes",          # 1-6 scale
    "capillary_refill",          # 1=<3sec, 2=>=3sec
    "pain",                      # 1=none … 5=extreme
    "peristalsis",               # 1=hypermotile … 4=absent
    "abdominal_distension",      # 1=none … 4=severe
    "nasogastric_tube",          # 1=none, 2=slight, 3=significant
    "nasogastric_reflux",        # 1=none, 2=>1L, 3=<1L
    "nasogastric_reflux_ph",
    "rectal_exam_feces",         # 1=normal … 4=increased
    "abdomen",                   # 1=normal … 5=distended_large
    "packed_cell_volume",
    "total_protein",
    "abdomcentesis_appearance",  # 1=clear, 2=cloudy, 3=serosanguinous
    "abdomcentesis_total_protein",
]


def load_colic_data(filepath):
    """Load tab-separated horse colic data."""
    X, y = [], []
    with open(filepath) as fh:
        for line in fh.readlines():
            parts = line.strip().split("\t")
            X.append([float(v) for v in parts[:-1]])
            y.append(float(parts[-1]))
    return np.array(X), np.array(y)


X_train, y_train = load_colic_data("horseColicTraining.txt")
X_test,  y_test  = load_colic_data("horseColicTest.txt")

print(f"Training : {X_train.shape[0]} samples, {X_train.shape[1]} features")
print(f"Test     : {X_test.shape[0]} samples")
print(f"Survival rate — train: {y_train.mean()*100:.1f}%  test: {y_test.mean()*100:.1f}%")

### 3.2 Correlation matrix — selecting interaction terms

The correlation matrix identifies which feature pairs have the strongest linear relationships.  
These pairs are the best candidates for interaction terms ($x_i \times x_j$), which allow the model to capture joint effects.

In [ ]:
def plot_correlation_matrix(X, names, title="Feature Correlation Matrix"):
    """Display correlation heatmap."""
    corr = np.corrcoef(X.T)
    fig, ax = plt.subplots(figsize=(11, 9))
    im = ax.imshow(corr, cmap="RdBu_r", vmin=-1, vmax=1)
    plt.colorbar(im, ax=ax, shrink=0.8)
    ax.set_xticks(range(len(names))); ax.set_xticklabels(names, rotation=90, fontsize=7)
    ax.set_yticks(range(len(names))); ax.set_yticklabels(names, fontsize=7)
    ax.set_title(title, fontsize=12)
    plt.tight_layout(); plt.show()
    return corr


def top_correlated_pairs(X, names, top_n=5):
    """Return the top-N most correlated (by |r|) feature pairs."""
    corr = np.corrcoef(X.T)
    pairs = [(i, j, corr[i, j])
             for i in range(corr.shape[0])
             for j in range(i+1, corr.shape[0])]
    pairs.sort(key=lambda t: abs(t[2]), reverse=True)
    print(f"\nTop {top_n} correlated pairs (interaction term candidates):")
    print(f"  {'Feature A':<26} {'Feature B':<26} {'r':>8}")
    for i, j, r in pairs[:top_n]:
        print(f"  {names[i]:<26} {names[j]:<26} {r:>8.4f}")
    return pairs[:top_n]


corr_matrix = plot_correlation_matrix(X_train, FEATURE_NAMES,
                                       "Feature Correlation Matrix (Training Set)")
top_pairs = top_correlated_pairs(X_train, FEATURE_NAMES, top_n=5)

### 3.3 Add interaction terms

In [ ]:
def add_interaction_terms(X, pairs):
    """Append x_i * x_j columns for each (i, j) pair."""
    cols = [X[:, i] * X[:, j] for i, j, *_ in pairs]
    return np.hstack([X] + [c.reshape(-1, 1) for c in cols])


# Use top-3 most correlated pairs as interaction terms
interaction_pairs = [(i, j) for i, j, _ in top_pairs[:3]]
interaction_labels = [(FEATURE_NAMES[i], FEATURE_NAMES[j]) for i, j in interaction_pairs]

X_train_aug = add_interaction_terms(X_train, interaction_pairs)
X_test_aug  = add_interaction_terms(X_test,  interaction_pairs)

print(f"Interaction terms added: {interaction_labels}")
print(f"Feature count: {X_train.shape[1]} → {X_train_aug.shape[1]}")

### 3.4 Feature standardization (no data leakage)

In [ ]:
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train_aug)   # fit only on training set
X_test_s  = scaler.transform(X_test_aug)        # apply same transform to test

print("Standardization complete.")
print(f"Train feature means (first 5): {X_train_s.mean(axis=0)[:5].round(6)}")
print(f"Train feature stds  (first 5): {X_train_s.std(axis=0)[:5].round(6)}")

---
## Part 4 — Model A: Custom Stochastic Gradient Ascent

Uses a **decaying learning rate** $\alpha = \frac{4}{1 + j + i} + 0.01$ to prevent oscillation near convergence.

In [ ]:
def stochastic_gradient_ascent(X, y, num_iterations=500, seed=42):
    """Stochastic gradient ascent with decaying learning rate.

    At each iteration, all samples are visited exactly once in random order.
    Learning rate alpha = 4/(1+j+i) + 0.01 decreases over time.
    """
    random.seed(seed)
    m, n = X.shape
    weights = np.ones(n)
    loss_history = []

    for j in range(num_iterations):
        data_index = list(range(m))
        for i in range(m):
            alpha = 4.0 / (1.0 + j + i) + 0.01      # decaying learning rate
            rand_idx = int(random.uniform(0, len(data_index)))
            h = sigmoid(np.dot(X[data_index[rand_idx]], weights))
            error = y[data_index[rand_idx]] - h
            weights += alpha * error * X[data_index[rand_idx]]
            del data_index[rand_idx]

        # Record log-loss after each complete pass
        probs = sigmoid(X @ weights)
        loss_history.append(log_loss(y, probs))

    return weights, loss_history


print("Training custom stochastic gradient ascent (500 iterations)...")
weights_sgd, loss_hist_sgd = stochastic_gradient_ascent(X_train_s, y_train, num_iterations=500)
print("Done.")

In [ ]:
plt.figure(figsize=(7, 3))
plt.plot(loss_hist_sgd, color='steelblue', lw=1.5)
plt.xlabel('Iteration (full pass)'); plt.ylabel('Log-Loss')
plt.title('Convergence — Custom Stochastic Gradient Ascent')
plt.tight_layout(); plt.show()

In [ ]:
y_pred_sgd = (sigmoid(X_test_s @ weights_sgd) >= 0.5).astype(float)
y_prob_sgd = sigmoid(X_test_s @ weights_sgd)

metrics_sgd = evaluate(y_test, y_pred_sgd, y_prob_sgd,
                       model_name="Custom Stochastic Gradient Ascent",
                       class_names=["Did Not Survive", "Survived"])

---
## Part 5 — Model B: Sklearn LogisticRegression (L2, lbfgs)

Uses the L-BFGS second-order solver with L2 regularization as a benchmark.

In [ ]:
clf = LogisticRegression(solver="lbfgs", max_iter=2000, C=1.0, random_state=42)
clf.fit(X_train_s, y_train)

y_pred_lr = clf.predict(X_test_s)
y_prob_lr = clf.predict_proba(X_test_s)[:, 1]

metrics_lr = evaluate(y_test, y_pred_lr, y_prob_lr,
                      model_name="Sklearn LogisticRegression (L2, lbfgs)",
                      class_names=["Did Not Survive", "Survived"])

### 5.1 ROC curve comparison

In [ ]:
plt.figure(figsize=(6, 5))
for name, y_prob in [("Custom SGD", y_prob_sgd), ("Sklearn LR", y_prob_lr)]:
    fpr, tpr, _ = roc_curve(y_test, y_prob)
    auc_val = roc_auc_score(y_test, y_prob)
    plt.plot(fpr, tpr, lw=2, label=f"{name} (AUC={auc_val:.3f})")
plt.plot([0,1],[0,1],'k--', lw=1, label='Random')
plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
plt.title('ROC Curves — Model Comparison')
plt.legend(loc='lower right')
plt.tight_layout(); plt.show()

---
## Part 6 — Stratified K-Fold Cross-Validation

Provides a robust generalisation estimate by training and evaluating on 5 different data splits.  
Standardization is applied **within each fold** to prevent data leakage.

In [ ]:
def stratified_kfold_cv(X, y, n_splits=5, C=1.0):
    """Stratified k-fold CV with per-fold standardization."""
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    fold_metrics = []

    for fold, (tr_idx, val_idx) in enumerate(skf.split(X, y), 1):
        X_tr, X_val = X[tr_idx], X[val_idx]
        y_tr, y_val = y[tr_idx], y[val_idx]

        # Fit scaler on fold-train only
        sc = StandardScaler()
        X_tr_s  = sc.fit_transform(X_tr)
        X_val_s = sc.transform(X_val)

        model = LogisticRegression(solver="lbfgs", max_iter=2000, C=C, random_state=42)
        model.fit(X_tr_s, y_tr)

        yp  = model.predict(X_val_s)
        ypr = model.predict_proba(X_val_s)[:, 1]

        fold_metrics.append({
            "accuracy" : (yp == y_val).mean(),
            "auc_roc"  : roc_auc_score(y_val, ypr),
            "log_loss" : log_loss(y_val, ypr),
            "f1"       : 2*((yp==1)&(y_val==1)).sum() / max((yp==1).sum()+(y_val==1).sum(), 1),
            "mcc"      : matthews_corrcoef(y_val, yp),
        })
        print(f"  Fold {fold}: acc={fold_metrics[-1]['accuracy']:.3f}  "
              f"auc={fold_metrics[-1]['auc_roc']:.3f}  "
              f"ll={fold_metrics[-1]['log_loss']:.3f}")

    print(f"\n{'='*50}")
    print(f"  {n_splits}-Fold CV Summary (mean ± std)")
    print(f"{'='*50}")
    for key in fold_metrics[0]:
        vals = [m[key] for m in fold_metrics]
        print(f"  {key:<12}: {np.mean(vals):.4f} ± {np.std(vals):.4f}")
    return fold_metrics


# Combine train+test for full CV
X_full = np.vstack([X_train_aug, X_test_aug])
y_full = np.concatenate([y_train, y_test])

print("Running 5-fold CV on full dataset (augmented with interaction terms)...\n")
cv_results = stratified_kfold_cv(X_full, y_full, n_splits=5)

---
## Part 7 — Summary Comparison Table

In [ ]:
print(f"\n{'='*60}")
print("  Final Test-Set Comparison")
print(f"{'='*60}")
print(f"  {'Metric':<14} {'Custom SGD':>14} {'Sklearn LR':>14}")
print(f"  {'-'*44}")
for key in ["accuracy", "precision", "recall", "specificity",
            "f1", "auc_roc", "log_loss", "mcc"]:
    print(f"  {key:<14} {metrics_sgd[key]:>14.4f} {metrics_lr[key]:>14.4f}")

print(f"\n  Note: error rate = 1 - accuracy")
print(f"  Custom SGD error rate : {(1-metrics_sgd['accuracy'])*100:.2f}%")
print(f"  Sklearn LR error rate : {(1-metrics_lr['accuracy'])*100:.2f}%")